In [2]:
from model_persistant_state import ModelPersistantState 

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. load a model
model_path = "models/mamba"
model_persistant_state = ModelPersistantState(model_path)

In [ ]:
model = model_persistant_state.load_model()